In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

tf.keras.backend.set_floatx('float32')

#Number of grid points along x and y axis
Nx = 101
Ny = 101

#domain size
xmin = 0.0
xmax = 1.0
ymin = 0.0
ymax = 1.0

#grid size
hx = (xmax - xmin)/(Nx - 1)
hy = (ymax - ymin)/(Ny - 1)

#weight 
omega = 0.1

In [ ]:
#This function considers all the inner points and organize the nodal values for the calculation of e
#input_shape = (Nx, Ny)
#output_shape = (number of edges, 2)
def organize_interaction(A):

    first = tf.concat([tf.reshape(A[1:, 1:-1], (-1, 1)), tf.reshape(A[0:-1, 1:-1], (-1, 1))], axis=1)
    
    second = tf.concat([tf.reshape(A[1:-1, 1:], (-1, 1)), tf.reshape(A[1:-1, 0:-1], (-1, 1))], axis=1)

    input = tf.concat([first, second], axis=0)
    
    return input

In [ ]:
#Multiplying the result of f(R) with PM matrix gives c_{i,j}
def calc_PM(Nx, Ny):
    #number of division along x and y axes
    m = Nx - 1
    n = Ny - 1

    num_interactions = (n-1)*m + (m-1)*n
    num_inner_nodes = (m-1)*(n-1)
    
    first_gap = n-2
    second_gap = (n-2) + (n-1)*(m-2)
    
    indices = np.zeros((4*num_inner_nodes, 2))
    values = np.zeros((4*num_inner_nodes, ))
    dense_size = np.array([num_inner_nodes, num_interactions]) 
    
    current = 0
    for i in range(num_inner_nodes):
        indices[current][:] = np.array([i, i])
        values[current] = 1.
        current = current + 1
        
        indices[current][:] = np.array([i, i+1+first_gap])
        values[current] = -1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap + int(i/(n-1))])
        values[current] = 1.
        current = current + 1

        indices[current][:] = np.array([i, i+1+first_gap+1+second_gap+1 + int(i/(n-1))])
        values[current] = -1.
        current = current + 1

    values = tf.convert_to_tensor(values, dtype='float32')
    
    PM = tf.sparse.SparseTensor(indices, values, dense_size)
    
    return PM

In [ ]:
def create_model(input_shape, nhu=2, npl=32):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(input_shape))

    for _ in range(nhu):
        model.add(tf.keras.layers.Dense(npl, 
                                        activation='relu',
                                        #kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))
                                        kernel_initializer='glorot_normal'))
    
    model.add(tf.keras.layers.Dense(1, activation='tanh'))

    return model

In [ ]:
#Reshapes H into (Nx-2)*(Ny-2) matrix and add 0s for the boundary
def reshape_H(Nx, Ny, A):
    A = tf.reshape(A, (Nx-2, Ny-2))

    #top and bottom in the xy plane
    top_and_bottom = tf.zeros((Nx-2, 1), dtype='float32')

    A = tf.concat([top_and_bottom, A, top_and_bottom], axis=1)

    left_and_right = tf.zeros((1, Ny), dtype='float32')

    A = tf.concat([left_and_right, A, left_and_right], axis=0)

    return A

In [ ]:
#takes the values of inner grid points and add boundary values
def update_boundary(Nx, Ny, H):

    #different values for different bounday points
    x = np.linspace(xmin, xmax, Nx)
    y = np.linspace(ymin, ymax, Ny)

    T1 = np.sin(5*np.pi*y/2)
    T2 = -x + 1
    T3 = -4*(y - 1/2)**2 + 1
    T4 = 0*x

    #top and bottom in the xy plane
    bottom = tf.convert_to_tensor(np.reshape(T4[1:-1], (Nx-2, 1)), dtype='float32')
    top = tf.convert_to_tensor(np.reshape(T2[1:-1], (Nx-2, 1)), dtype='float32')
    
    output = tf.concat([bottom, H, top], axis=1)
    
    left = tf.convert_to_tensor(np.reshape(T1, (1, Ny)), dtype='float32')
    right = tf.convert_to_tensor(np.reshape(T3, (1, Ny)), dtype='float32')
    
    output = tf.concat([left, output, right], axis=0)

    return output


In [ ]:
#Energy Functional
def compute_energy(phi):
    #combination of forward and backward
    grad_f = tf.math.square(phi[2:, 1:-1] - phi[1:-1, 1:-1])/(hx*hx) \
              + tf.math.square(phi[1:-1, 2:] - phi[1:-1, 1:-1])/(hy*hy)
    
    grad_b = tf.math.square(phi[1:-1, 1:-1] - phi[0:-2, 1:-1])/(hx*hx) \
              + tf.math.square(phi[1:-1, 1:-1] - phi[1:-1, 0:-2])/(hy*hy)
            
    grad2 = (grad_f + grad_b)/2.
    
    integral = 0.5*grad2

    #riemann sum
    energy = tf.reduce_sum(hx*hy*integral)
    
    return energy


In [ ]:
#Algorithm 2 from the paper
#M is the number of message passing steps
#fR and fO are the networks
def march_forward(M, phi_in, fR, fO, PM):

    Nx = phi_in.shape[0]
    Ny = phi_in.shape[1]

    H = tf.zeros(phi_in.shape, dtype='float32')
    
    for _ in range(M):
        input_phi = organize_interaction(phi_in)
        input_H = organize_interaction(H)
        input1 = tf.concat([input_phi, input_H], axis=1)

        
        output1 = fR(input1)

        c = tf.sparse.sparse_dense_matmul(PM, output1)
        
        input2 = tf.concat((tf.reshape(phi_in[1:-1, 1:-1], (-1, 1)), c), axis=1)
        
        output2 = fO(input2)
        
        H = reshape_H(Nx, Ny, output2)
    
    #adding the boundary values
    phi_out = update_boundary(Nx, Ny, H[1:-1, 1:-1])
    return phi_out, c 

In [ ]:
def compute_loss(M, phi_in, fR, fO, PM, initial=False, direct_training=False):
    phi_out, _ = march_forward(M, phi_in, fR, fO, PM)
    
    energy_in = compute_energy(phi_in)
    energy_out = compute_energy(phi_out)
    
    if initial:
        loss = tf.reduce_mean(tf.math.square(phi_in - phi_out))
    elif direct_training:
        loss = energy_out
    else:
        loss = tf.math.reduce_mean(tf.math.square(phi_out - phi_in)) + omega*energy_out/energy_in
    
    return energy_in, energy_out, loss

In [ ]:
def initialize_phi(Nx, Ny):

    phi = np.zeros((Nx, Ny))

    phi[1:-1, 1:-1] = np.random.uniform(0, 1, (Nx-2, Ny-2))

    #different values for different bounday points
    x = np.linspace(xmin, xmax, Nx)
    y = np.linspace(ymin, ymax, Ny)

    T1 = np.sin(5*np.pi*y/2)
    T2 = -x + 1
    T3 = -4*(y - 1/2)**2 + 1
    T4 = 0*x

    phi[:, 0]       = T4
    phi[:, Ny-1]    = T3
    phi[0, :]       = T1
    phi[Nx-1, :]    = T2

    return tf.convert_to_tensor(phi, dtype='float32')

In [ ]:
class FONN():
    def __init__(self, M):
        self.M = M 
        self.PM = calc_PM(Nx, Ny)
        
        input_shape = (4, )

        self.fR = create_model(input_shape=input_shape)
        self.fO = create_model(input_shape=(2, ))
        
        self.phi_0 = initialize_phi(Nx, Ny)
        self.phi   = initialize_phi(Nx, Ny)
    
    def initial_training(self, epochs):
        optimizer = tf.keras.optimizers.Adam()

        for ep in range(epochs):
            noise = tf.random.normal(self.phi_0.shape, mean=0.0, stddev=0.05, dtype='float32')
            noisy_input = self.phi_0 + noise

            with tf.GradientTape() as tape:
                _, _, loss = compute_loss(self.M, noisy_input, self.fR, self.fO, self.PM, initial=True)
            grads = tape.gradient(loss, tape.watched_variables())
            
            optimizer.apply_gradients(zip(grads, tape.watched_variables()))
            del tape

            print("initial training step = ", ep, " loss = ", loss.numpy())

    #This function trains the networks using energy itself as the loss function
    def direct_training(self, steps, save_plots=False, location=""):
        optimizer = tf.keras.optimizers.Adam()

        if save_plots:
            name_Loss = location + "/loss.dat" 
            file_Loss = open(name_Loss, "w")
        
        for step in range(steps):
            with tf.GradientTape() as tape:
                energy_in, _, loss = compute_loss(self.M, self.phi, self.fR, self.fO, self.PM, direct_training=True)

            grads = tape.gradient(loss, tape.watched_variables())

            optimizer.apply_gradients(zip(grads, tape.watched_variables()))
            del tape

            print("step = ", step, " loss = ", loss.numpy())

            if save_plots:
                result_E = str(loss.numpy()) + "\n"
                file_Loss.write(result_E)
        
        file_Loss.close()

        output, _ = march_forward(self.M, self.phi, self.fR, self.fO, self.PM)
        E_output = compute_energy(output)

        if save_plots:
            data_file_name = location + "/initial.txt"
            np.savetxt(data_file_name, np.reshape(self.phi.numpy(), -1))

            data_file_name = location + "/final.txt"
            np.savetxt(data_file_name, np.reshape(output.numpy(), -1))
            
            fig_name = location + "/initial.pdf"
            title_ = "$E(\phi)$ = " + str(energy_in.numpy())
            plt.figure()
            plt.contourf(tf.transpose(self.phi))
            plt.colorbar()
            plt.title(title_)
            plt.savefig(fig_name)
            plt.close()

            fig_name = location + "/final.pdf"
            title_ = "$E(\phi)$ = " + str(E_output.numpy())
            plt.figure()
            plt.contourf(tf.transpose(output))
            plt.colorbar()
            plt.title(title_)
            plt.savefig(fig_name)
            plt.close()

    
    #K -> fine tuning steps
    def progressive_method(self, steps, K, save_plots=False, save_fRfO = False, location=""):
        optimizer = tf.keras.optimizers.Adam()
        
        #open the files
        if save_plots:
            name_E = location + "/energy.txt"
            file_E = open(name_E, "w")

            name_Loss = location + "/loss.txt"
            file_Loss = open(name_Loss, "w")

        for step in range(steps):
            
            for k in range(K):
                with tf.GradientTape() as tape:
                    energy_in, energy_out, loss = compute_loss(self.M, self.phi, self.fR, self.fO, self.PM)
                
                grads = tape.gradient(loss, tape.watched_variables())

                optimizer.apply_gradients(zip(grads, tape.watched_variables()))
                del tape

                if save_plots:
                    result_Loss = str(loss.numpy()) + "\n"
                    file_Loss.write(result_Loss)
                        
            self.phi, cij = march_forward(self.M, self.phi, self.fR, self.fO, self.PM)

            print("step = ", step, " energy_in = ", energy_in.numpy(), " energy_out = ", energy_out.numpy())
            

            #To save plots and result
            if save_plots:

                result_E = str(energy_in.numpy()) + "\t" + str(energy_out.numpy()) + "\n"
                file_E.write(result_E)
                
                #save figure
                if step % 50 == 0:
                    data_file_name = location + "/step_" + str(step) + ".txt"
                    np.savetxt(data_file_name, np.reshape(self.phi.numpy(), -1))
                    fig_name = location + "/" + str(step) + ".pdf"
                    title_ = "$E(u)$ = " + str(compute_energy(self.phi).numpy())
                    plt.figure()
                    plt.contourf(tf.transpose(self.phi))
                    plt.colorbar()
                    plt.title(title_)
                    plt.savefig(fig_name)
                    plt.close()
                        
                    #plot cij
                    data_file_name = location + "/cij_" + str(step) + ".txt"
                    np.savetxt(data_file_name, cij.numpy())
                    fig_name = location + "/cij_" + str(step) + ".pdf"
                    plt.figure()
                    plt.contourf(np.transpose(np.reshape(cij.numpy(), (Nx-2, Ny-2))))
                    plt.colorbar()
                    plt.savefig(fig_name)
                    plt.close()

                    if save_fRfO:
                        self.fR.save(location + "/model_fR_" + str(step) + ".h5")
                        self.fO.save(location + "/model_fO_" + str(step) + ".h5")
                
        if save_plots:
            file_E.close()
            file_Loss.close()
            
            E = np.genfromtxt(name_E, dtype='float32')
            plt.figure()
            plt.semilogy(E[:,0], linewidth=2)
            plt.grid(visible=True, which='both')
            plt.title("$E(\phi)$")
            plt.savefig(location + "/E.pdf")
            plt.close()

In [ ]:
#M = 1
model = FONN(1)


location = "./Heat_Results"
if not os.path.exists(location):
    os.makedirs(location)


In [ ]:
model.initial_training(1000)

In [ ]:
model.progressive_method(steps=501, K=10, save_plots=True, location=location)